# GWpy Tutorial

GWpy is a collaboration-driven Python package providing tools for studying data from ground-based gravitational-wave detectors.

GWpy provides a user-friendly, intuitive interface to the common time-domain and frequency-domain data produced by the LIGO and Virgo instruments and their analysis, with easy-to-follow tutorials at each step.

- https://gwpy.github.io/docs/v0.1/timeseries/index.html
- https://www.kaggle.com/mistag/data-preprocessing-with-gwpy

In [ ]:
!python -m pip install gwpy
!pip install astropy==4.2.1

In [ ]:
import numpy as np

from gwpy.timeseries import TimeSeries
from gwpy.plot import Plot
from scipy import signal
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

In [ ]:
from gwpy.timeseries import TimeSeries

data = TimeSeries.fetch_open_data('L1', 1126259446, 1126259478)
data

In [ ]:
def read_file(fname):
    data = np.load(fname)
    data = rescale(data)
    d1 = TimeSeries(data[0,:], sample_rate=2048)
    d2 = TimeSeries(data[1,:], sample_rate=2048)
    d3 = TimeSeries(data[2,:], sample_rate=2048)  
    return d1, d2, d3


def rescale(data):
    # Normalize to [-1,1]
    data = (data - np.mean(data, axis=1).reshape(3,-1)) / (np.max(data, axis=1) - np.min(data, axis=1)).reshape(3,-1) * 2
    return data

    
def plot_time_data(d1, d2, d3, title=''):
    plot = Plot(d1, d2, d3, separate=True, sharex=True, figsize=[12, 8])
    ax = plot.gca()   
    ax.set_title(title)
    ax.set_xlim(0,2)
    ax.set_xlabel('Time [s]')
    plot.show()
        
        
def preprocess(d1, d2, d3, bandpass=False, normalize=True, lf=35, hf=350):
    white_d1 = d1.whiten(window=("tukey",0.2))
    white_d2 = d2.whiten(window=("tukey",0.2))
    white_d3 = d3.whiten(window=("tukey",0.2))
        
    if bandpass: # bandpass filter
        bp_d1 = white_d1.bandpass(lf, hf) 
        bp_d2 = white_d2.bandpass(lf, hf)
        bp_d3 = white_d3.bandpass(lf, hf)
        return bp_d1, bp_d2, bp_d3
    else: # only whiten
        return white_d1, white_d2, white_d3
    
sample_signals    = read_file('/kaggle/input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy')
sample_no_signals = read_file('/kaggle/input/g2net-gravitational-wave-detection/train/0/0/0/00001f4945.npy')

In [ ]:
plot_time_data(*sample_signals, title='raw')

In [ ]:
plot_time_data(*preprocess(*sample_signals, bandpass=False), title='whiten')

In [ ]:
plot_time_data(*preprocess(*sample_signals, bandpass=True),  title='bandpass')

In [ ]:
np.array( sample_signals[0].to_value().shape ) 

# Q-Transform

In [ ]:
r1, r2, r3 = read_file('../input/g2net-gravitational-wave-detection/train/0/0/0/000a5b6e5c.npy') # this signal has target=1
p1, p2, p3 = preprocess(r1, r2, r3)
hq = p2.q_transform(qrange=(16,32), frange=(30,400), logf=True, whiten=False)
fig4 = hq.plot(figsize=[12, 10])
ax = fig4.gca()
fig4.colorbar(label="Normalised energy")
ax.grid(False)
ax.set_yscale('log')
ax.set_xlabel('Time [s]');

In [ ]:
Q_RANGE = (16,32)
F_RANGE = (30,400)

def create_rgb(fname):
    r1, r2, r3 = read_file(fname)
    p1, p2, p3 = preprocess(r1, r2, r3)
    hq1 = p1.q_transform(qrange=Q_RANGE, frange=F_RANGE, logf=True, whiten=False)
    hq2 = p2.q_transform(qrange=Q_RANGE, frange=F_RANGE, logf=True, whiten=False)
    hq3 = p3.q_transform(qrange=Q_RANGE, frange=F_RANGE, logf=True, whiten=False)
    img = np.zeros([hq1.shape[0], hq1.shape[1], 3], dtype=np.uint8)
    scaler = MinMaxScaler()
    img[:,:,0] = 255*scaler.fit_transform(hq1)
    img[:,:,1] = 255*scaler.fit_transform(hq2)
    img[:,:,2] = 255*scaler.fit_transform(hq3)
    return Image.fromarray(img).rotate(90, expand=1).resize((760,760))

In [ ]:
create_rgb('../input/g2net-gravitational-wave-detection/train/0/0/0/000a5b6e5c.npy')

# Signal Processing

https://gwpy.github.io/docs/v0.1/signal/index.html

## rayleigh_spectrum([fftlength, overlap])	

https://gwpy.github.io/docs/stable/examples/spectrogram/rayleigh.html

In [ ]:
r1, r2, r3 = read_file('../input/g2net-gravitational-wave-detection/train/0/0/0/000a5b6e5c.npy') # this signal has target=1
p1, p2, p3 = preprocess(r1, r2, r3)
hq = p2.rayleigh_spectrogram(2)
hq
fig4 = hq.plot(figsize=[12, 10])
ax = fig4.gca()
fig4.colorbar(label="Rayleigh Spectrogram")
ax.grid(False)
ax.set_yscale('log')
ax.set_xlabel('Time [s]');

In [ ]:
## Source: https://gwpy.github.io/docs/stable/examples/spectrogram/rayleigh.html
rayleigh = p2.rayleigh_spectrogram(stride=.5, fftlength=.2, overlap=.1)
print(rayleigh)

plot = rayleigh.plot(norm='log', vmin=0.25, vmax=4)
ax = plot.gca()
ax.set_yscale('log')
ax.set_ylim(30, 1500)
ax.set_title('Sensitivity of LIGO-Livingston around GW151226')
ax.colorbar(cmap='coolwarm', label='Rayleigh statistic')
plot.show()

In [ ]:
## Coherence

In [ ]:
r1, r2, r3 = read_file('../input/g2net-gravitational-wave-detection/train/0/0/0/000a5b6e5c.npy') # this signal has target=1
p1, p2, p3 = preprocess(r1, r2, r3)
hq = p1.coherence(p2) # + p2.coherence(p3, 2) + p3.coherence(p1, 2)
hq
# fig4 = hq.plot(figsize=[12, 100])
# ax = fig4.gca()
# fig4.colorbar(label="Rayleigh Spectrogram")
# ax.grid(False)
# ax.set_yscale('log')
# ax.set_xlabel('Time [s]');

## FrequencySeries Spectral Density

A measurement of the acceleration spectral density (ASD) is the usual way to specify random vibration. The root mean square acceleration (Grms) is the square root of the area under the ASD curve in the frequency domain. The Grms value is typically used to express the overall energy of a particular random vibration event and is a statistical value used in mechanical engineering for structural design and analysis purposes.

While the term power spectral density (PSD) is commonly used to specify a random vibration event, ASD is more appropriate when acceleration is being measured and used in structural analysis and testing.

- https://gwpy.github.io/docs/v0.1/timeseries/index.html#gwpy.timeseries.TimeSeries.psd
- https://en.wikipedia.org/wiki/Random_vibration

In [ ]:
from gwpy.timeseries import TimeSeries
gwdata = p2
# gwdata = TimeSeries.fetch('H1:LDAS-STRAIN', 'September 16 2010 06:40', 'September 16 2010 06:50')
spectrum = gwdata.asd(fftlength=.2, overlap=.1)
plot = spectrum.plot()
ax = plot.gca()
# ax.set_xlim(40, 4000)
ax.set_ylabel(r'Gravitational-wave strain ASD [strain$/\sqrt{\mathrm{Hz}}$]')
# ax.set_ylim(1e-23, 1e-19)
plot.show()

In [ ]:
from gwpy.timeseries import TimeSeries
gwdata = p2
# gwdata = TimeSeries.fetch('H1:LDAS-STRAIN', 'September 16 2010 06:40', 'September 16 2010 06:50')
spectrum = gwdata.psd(fftlength=.2, overlap=.1)
plot = spectrum.plot()
ax = plot.gca()
# ax.set_xlim(40, 4000)
ax.set_ylabel(r'Gravitational-wave strain PSD [strain$/\sqrt{\mathrm{Hz}}$]')
# ax.set_ylim(1e-23, 1e-19)
plot.show()